<a href="https://colab.research.google.com/github/ghenwafkh/AI-and-machine-learning/blob/master/ColBERT_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install colbert


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 10.4 MB/s eta 0:00:00
  Created wheel for colbert: filename=Colbert-0.30-py3-none-any.whl size=70358 sha256=8b395e47e8b02ce474e6ba1f4bc4bd9bcd94f753a6def290604a92ec3ec8b854
  Stored in directory: /root/.cache/pip/wheels/a9/be/45/ab937e99a6976af70dbc3233c3e8efa17734303cc91443d9de
Successfully built colbert


In [12]:
pip install git+https://github.com/stanford-futuredata/ColBERT.git


  Cloning https://github.com/stanford-futuredata/ColBERT.git to /tmp/pip-req-build-0222rebt
  Running command git clone --filter=blob:none --quiet https://github.com/stanford-futuredata/ColBERT.git /tmp/pip-req-build-0222rebt
  Resolved https://github.com/stanford-futuredata/ColBERT.git to commit b53d8bbe6e83deeb7c76fc6b912da37554981a02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [14]:
import os
import sys
sys.path.insert(0, '../')
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import indexer, searcher

In [15]:
!mkdir -p downloads/

# ColBERTv2 checkpoint trained on MS MARCO Passage Ranking (388MB compressed)
!wget https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz -P downloads/
!tar -xvzf downloads/colbertv2.0.tar.gz -C downloads/

# The LoTTE dev and test sets (3.4GB compressed)
!wget https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/lotte.tar.gz -P downloads/
!tar -xvzf downloads/lotte.tar.gz -C downloads/

--2023-11-29 14:14:26--  https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405924985 (387M) [application/octet-stream]
Saving to: ‘downloads/colbertv2.0.tar.gz’

colbertv2.0.tar.gz  100%[===================>] 387.12M  5.13MB/s    in 72s     

2023-11-29 14:15:38 (5.36 MB/s) - ‘downloads/colbertv2.0.tar.gz’ saved [405924985/405924985]

colbertv2.0/
colbertv2.0/artifact.metadata
colbertv2.0/vocab.txt
colbertv2.0/tokenizer.json
colbertv2.0/special_tokens_map.json
colbertv2.0/tokenizer_config.json
colbertv2.0/config.json
colbertv2.0/pytorch_model.bin
--2023-11-29 14:15:45--  https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/lotte.tar.gz
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Con

In [16]:
dataroot = 'downloads/lotte'
dataset = 'lifestyle'
datasplit = 'dev'

queries = os.path.join(dataroot, dataset, datasplit, 'questions.search.tsv')
collection = os.path.join(dataroot, dataset, datasplit, 'collection.tsv')

queries = Queries(path=queries)
collection = Collection(path=collection)

f'Loaded {len(queries)} queries and {len(collection):,} passages'


[Nov 29, 14:32:24] #> Loading the queries from downloads/lotte/lifestyle/dev/questions.search.tsv ...
[Nov 29, 14:32:24] #> Got 417 queries. All QIDs are unique.

[Nov 29, 14:32:24] #> Loading collection...
0M 


'Loaded 417 queries and 268,893 passages'

In [17]:
print(queries[24])
print()
print(collection[89852])
print()

are blossom end rot tomatoes edible?

I don't know what J-Rock is definitively as it could mean a number of things, however if you mean a 'metal' style rock guitar sound that I believe to be popular over there, then it can be quite simple: Strip all of your pedals and effects, and turn all dials to zero; Turn the volume up as far as it will go; Keep the Bass and Treble dials high; Turn down any 'Middle' dials until the sound becomes scooped - bear in the mind that middle frequencies are important for clarity, so find a decent compromise; Bring up the gain until you get the sound you require; Finally, bring the volume down to a more manageable level. This will get you a lot of the way, but without any clearer description of what you want to sound like, then I can't offer any more help.



In [18]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300   # truncate passages at 300 tokens

checkpoint = 'downloads/colbertv2.0'
index_name = f'{dataset}.{datasplit}.{nbits}bits'

In [19]:
print(checkpoint)
print(index_name)

downloads/colbertv2.0
lifestyle.dev.2bits


In [ ]:
with Run().context(RunConfig(nranks=4, experiment='notebook')):  # nranks specifies the number of GPUs to use.
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits)

    indexer = indexer.Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection, overwrite=True)



[Nov 29, 14:50:15] #> Creating directory /content/experiments/notebook/indexes/lifestyle.dev.2bits 


#> Starting...
#> Starting...
#> Starting...
#> Starting...


In [ ]:
query = queries[37]   # or supply your own query

print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")